# 12.3 병든 잎사귀 식별 경진대회 베이스라인 모델
- [병든 잎사귀 식별 경진대회 링크](https://www.kaggle.com/c/plant-pathology-2020-fgvc7)
- [베이스라인 모델 코드 참고 링크](https://www.kaggle.com/akasharidas/plant-pathology-2020-in-pytorch)

## 12.3.1 시드값 고정 및 GPU 장비 설정
### 시드값 고정

In [1]:
%config Completer.use_jedi = False

In [2]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### GPU 장비 설정

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

## 12.3.2 데이터 준비

In [4]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 훈련 데이터, 검증 데이터 분리

In [5]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, test_size=0.1, stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                                random_state=50)

### 데이터셋 클래스 정의

In [6]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id + '.jpg'
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
        
        if self.is_test:
            return image
        else:
            label = np.argmax(self.df.iloc[idx, 1:5])
            return image, label
        

### 이미지 변환기 정의

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [8]:
transform_train = A.Compose([
    A.Resize(450, 650),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=30, p=0.3
    ),
    A.OneOf([A.Emboss(p=1),
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

In [9]:
transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),
    ToTensorV2()
])

### 데이터셋 및 데이터 로더 생성

In [10]:
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [11]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [12]:
from torch.utils.data import DataLoader

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

In [13]:
!pip install efficientnet-pytorch==0.7.1

In [14]:
from efficientnet_pytorch import EfficientNet

In [15]:
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)
model = model.to(device)

Loaded pretrained weights for efficientnet-b7


## 12.3.4 모델 훈련 및 성능 검증

### 손실 함수와 옵티마이저 설정

In [16]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

In [18]:
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm
epochs = 5

for epoch in range(epochs):
    # 훈련
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0 # 에폭별 손실값 초기화(훈련 데이터용)
    
    for images, labels in tqdm(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() # 옵티마이저 내 기울기 초기화
        outputs = model(images) # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        loss = criterion(outputs, labels) # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        epoch_train_loss += loss.item() # 현제 배치에서 손실 추가(훈련 데이터용)
        loss.backward() # 역전파 수행
        optimizer.step() # 가중치 갱신
    print(f'에폭 {epoch + 1}/{epochs} - 훈련 데이터 손실값: {epoch_train_loss / len(loader_train): .4f}')
    
    # 검증
    model.eval() # 모델을 평가 상태로 설정
    epoch_valid_loss = 0 # 에폭별 손실값 초기화(검증 데이터 용)
    preds_list = [] # 예측 확률값 저장용 리스트 초기화
    true_onehot_list = [] # 실제 타깃값 저장용 리스트 초기화
    
    with torch.no_grad(): # 기울기 계산 비활성화
        # 미니배치 단위로 검증
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy()
            true_onehot = torch.eye(4)[labels].cpu().numpy()

            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    print(f'에폭 {epoch + 1}/{epochs} - 검증 데이터 손실값: {epoch_valid_loss/len(loader_valid) : .4f}/ 검증 데이터 ROC_AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')

  0%|          | 0/410 [00:00<?, ?it/s]

에폭 1/5 - 훈련 데이터 손실값:  0.6953
에폭 1/5 - 검증 데이터 손실값:  0.2325/ 검증 데이터 ROC_AUC : 0.9676


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 2/5 - 훈련 데이터 손실값:  0.3465
에폭 2/5 - 검증 데이터 손실값:  0.2163/ 검증 데이터 ROC_AUC : 0.9800


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 3/5 - 훈련 데이터 손실값:  0.2324
에폭 3/5 - 검증 데이터 손실값:  0.1646/ 검증 데이터 ROC_AUC : 0.9834


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 4/5 - 훈련 데이터 손실값:  0.1613
에폭 4/5 - 검증 데이터 손실값:  0.2363/ 검증 데이터 ROC_AUC : 0.9792


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 5/5 - 훈련 데이터 손실값:  0.1556
에폭 5/5 - 검증 데이터 손실값:  0.1905/ 검증 데이터 ROC_AUC : 0.9659


## 12.3.5 예측 및 결과 제출

In [21]:
dataset_test = ImageDataset(test, img_dir=img_dir, transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

## 예측

In [22]:
model.eval()

preds = np.zeros((len(test), 4))

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
        preds[i*batch_size:(i+1)*batch_size] += preds_part

### 결과 제출

In [23]:
submission[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds
submission.to_csv('submission.csv', index=False)